**Import Necessari**

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

**Funzioni Utilizzate**

In [ ]:
def generator(dataset,batchsize,moves=10):
  while True:
    rand = np.random.randint(dataset.shape[0],size=batchsize)
    y_truth = dataset[rand]
    blurred = np.copy(y_truth)
    moving = tf.expand_dims(y_truth,axis=-1)

    for i in range(moves):
      #RandomTranslation requires a channel axis
      moving = layers.RandomTranslation(0.07,0.07,fill_mode='constant',interpolation='bilinear')(moving)
      blurred = blurred + tf.squeeze(moving)
    blurred = blurred/(moves+1)
    yield(blurred,y_truth)

def plot(x,y):
  no = x.shape[0]
  print(no)
  plt.figure(figsize=(2*no, 4))
  for i in range(no):
    # display original
    ax = plt.subplot(2, no, i + 1)
    plt.imshow(x[i])
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, no, i + 1 + no)
    plt.imshow(y[i])
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
  plt.show()

def mean_square_error(x,x_true):
  summation = 0  #variable to store the summation of differences
  n = len(x) #finding total number of items in list
  for i in range (0,n):  #looping through each element of the list
    difference = x[i] - x_true[i]  #finding the difference between observed and predicted value
    squared_difference = difference**2  #taking square of the differene 
    summation = summation + squared_difference  #taking a sum of all the differences
  MSE = summation/n  #dividing summation by total values to obtain average
  print ("The Mean Square Error is: " , MSE)
  return MSE

**Definizione del Modello**

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = (x_train/255.).astype(np.float32)
x_test = (x_test/255.).astype(np.float32)

**Fase di Training del Modello**

In [2]:
my_gen = generator(x_train,5)
test_x,test_y = next(my_gen)
plot(test_x,test_y)

NameError: name 'generator' is not defined

**Fase di Validazione del Modello**

In [ ]:
y = [11,20,19,17,10]
y_bar = [12,18,19.5,18,9]

mean_square_error(y,y_bar);
#mean_square_error(test_x,test_y);